In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pybaseball import statcast_batter, statcast_pitcher
from pybaseball import statcast_pitcher_percentile_ranks
from pybaseball import pitching_stats
from pybaseball import playerid_lookup, playerid_reverse_lookup
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import NormalPredictor, BaselineOnly
from surprise.prediction_algorithms import SVD, SVDpp, NMF
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV, cross_validate

#### Creating a function to pull pitcher names with over 75 plate appearances against in a given year, gathering their MLBAM IDs, and then pulling their pitch level data from the selected years

In [10]:
def fetch_pitcher_data_for_years(years):
    combined_df = pd.DataFrame()

    for year in years:
        # Fetch pitching stats for the given year
        pitching_stats_df = pitching_stats(year, qual=75)

        # Extract first and last names
        last_names = [name.split()[-1] for name in pitching_stats_df['Name']]
        first_names = [name.split()[0] for name in pitching_stats_df['Name']]
        name_tuples = zip(first_names, last_names)
        result = set(name_tuples)

        player_ids = []

        # Fetch player IDs
        for first_name, last_name in result:
            try:
                player_id_df = playerid_lookup(last_name, first_name)
                if player_id_df is not None and not player_id_df.empty:
                    player_ids.append(player_id_df)
            except Exception as e:
                print(f"Error fetching player ID for {first_name} {last_name}: {e}")

        # Combine individual DataFrames
        combined_player_ids = pd.concat(player_ids, ignore_index=True)['key_mlbam']

        # Fetch data for each pitcher
        for id in combined_player_ids:
            try:
                pitcher_df = statcast_pitcher(f'{year}-04-01', f'{year}-09-30', id)
                if pitcher_df is not None and not pitcher_df.empty:
                    combined_df = pd.concat([combined_df, pitcher_df])
            except Exception as e:
                print(f"Error fetching data for {id}: {e}")

    return combined_df

# Example usage
years = [2021, 2022, 2023] 
combined_data = fetch_pitcher_data_for_years(years)

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

In [12]:
combined_data.to_csv('data/pitch_data_21_22_23.zip', index=False)

combined_data = pd.read_csv('data/pitch_data_21_22_23.zip')

In [13]:
combined_data.shape

(982533, 92)

In [14]:
# Dropping rows with nulls in the 'events' column because we only want pitches that ended an at bat

combined_data['events'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 982533 entries, 0 to 982532
Series name: events
Non-Null Count   Dtype 
--------------   ----- 
253512 non-null  object
dtypes: object(1)
memory usage: 7.5+ MB


In [15]:
combined_data_filtered = combined_data.dropna(subset=['events'])
combined_data_filtered.shape

(253512, 92)

In [16]:
# Checking out the outcomes listed
combined_data_filtered['events'].unique()

array(['strikeout', 'single', 'field_error', 'field_out', 'home_run',
       'fielders_choice_out', 'double', 'force_out', 'sac_fly', 'walk',
       'triple', 'grounded_into_double_play', 'caught_stealing_2b',
       'hit_by_pitch', 'fielders_choice', 'sac_bunt', 'double_play',
       'catcher_interf', 'strikeout_double_play', 'sac_fly_double_play',
       'sac_bunt_double_play', 'other_out', 'pickoff_caught_stealing_3b',
       'caught_stealing_home', 'caught_stealing_3b', 'stolen_base_2b',
       'pickoff_2b', 'pickoff_1b', 'pickoff_caught_stealing_2b',
       'pickoff_3b', 'triple_play', 'pickoff_caught_stealing_home'],
      dtype=object)

In [18]:
# Keeping only events that will be used for our 'rating' for modeling
events_to_keep = ['home_run', 'walk', 'single', 'double', 'hit_by_pitch', 'sac_fly', 'field_out', 'strikeout', 'triple', 
                  'double_play', 'force_out', 'grounded_into_double_play', 'strikeout_double_play', 'fielders_choice', 
                  'fielders_choice_out', 'sac_fly_double_play', 'triple_play', 'field_error'  
                  ]
combined_data_filtered = combined_data_filtered[combined_data_filtered['events'].isin(events_to_keep)]

In [19]:
combined_data_filtered.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FS,2021-09-26,83.6,-2.65,5.67,"Gausman, Kevin",596115,592332,strikeout,swinging_strike,...,2,2,1,1,2,Standard,Standard,235.0,-0.087,-0.458
6,FF,2021-09-26,95.9,-2.51,5.88,"Gausman, Kevin",453568,592332,single,hit_into_play,...,2,2,1,1,2,Infield shift,Standard,216.0,0.045,0.239
12,FF,2021-09-26,95.5,-2.48,5.82,"Gausman, Kevin",663898,592332,strikeout,swinging_strike,...,2,2,1,1,2,Standard,Standard,215.0,-0.050,-0.213
16,FF,2021-09-26,95.7,-2.39,5.82,"Gausman, Kevin",641658,592332,field_error,hit_into_play,...,2,2,1,1,2,Standard,Standard,217.0,0.045,0.304
21,FS,2021-09-26,85.3,-2.64,5.72,"Gausman, Kevin",602074,592332,field_out,hit_into_play,...,2,2,1,1,2,Standard,Standard,223.0,-0.040,-0.174


In [20]:
# Checking out the frequency a given hitter appears in the new df
combined_data_filtered['batter'].value_counts()

batter
543760    1081
518692    1027
502671    1021
665489    1008
621566     985
          ... 
670237       1
685087       1
700840       1
457915       1
656302       1
Name: count, Length: 1300, dtype: int64

In [22]:
# Dropping rows where the batters have seen the pitcher fewer than 10 times
batter_counts = pd.DataFrame(combined_data_filtered['batter'].value_counts())
batter_counts = batter_counts.loc[batter_counts['count'] > 10]
combined_data_filtered = combined_data_filtered[combined_data_filtered['batter'].isin(batter_counts.index)]

In [23]:
# Filtering the df to only include relevant columns
cols = ['batter', 'pitcher', 'events']
combined_data_filtered = combined_data_filtered[cols]

In [24]:
combined_data_filtered

,batter,pitcher,events
0,596115,592332,strikeout
6,453568,592332,single
12,663898,592332,strikeout
16,641658,592332,field_error
21,602074,592332,field_out
...,...,...,...
982509,673962,573186,strikeout
982513,666969,573186,field_out
982514,663993,573186,single
982519,608369,573186,walk


### Now it's time to create our wOBA statistic, what we will be using for our 'rating' metric

In [27]:
# Assigning numeric values to each potential event
def woba_value(events):
    if events == 'home_run':
        return 2.014
    elif events == 'triple':
        return 1.575
    elif events == 'double':
        return 1.248
    elif events == 'single':
        return 0.855
    elif events == 'hit_by_pitch':
        return 0.727
    elif events == 'walk':
        return 0.697
    else:
        return 0    

combined_data_filtered['woba_value'] = combined_data_filtered['events'].apply(woba_value)

In [28]:
combined_data_filtered.groupby(['batter', 'pitcher']).sum()

events  woba_value
batter pitcher                                                               
405395 425794                         field_outfield_outfield_out       0.000
       425844   strikeoutfield_outfield_outstrikeoutfield_outf...       0.000
       448179                                   field_outhome_run       2.014
       453286                                        singlesingle       1.710
       456501                  strikeoutgrounded_into_double_play       0.000
...                                                           ...         ...
807799 676664                                  strikeoutfield_out       0.000
       680694   field_outfield_outsinglegrounded_into_double_play       0.855
       682243                            field_outdoublefield_out       1.248
       686610                    doublestrikeoutfield_outhome_run       3.262
       694297                                  strikeoutfield_out       0.000

[64918 rows x 2 columns]

In [29]:
woba_denom_df = combined_data_filtered.groupby(['batter', 'pitcher', 'events']).count()

In [30]:
# Define a function to determine if an event counts as an AB, BB, SF, or HBP
def categorize_event(event):
    if event in ['single', 'double', 'triple', 'home_run', 'field_out', 'strikeout', 'etc...']:  # add all events that count as AB
        return 'AB'
    elif event == 'walk':
        return 'BB'
    elif event == 'sac_fly':
        return 'SF'
    elif event == 'hit_by_pitch':
        return 'HBP'
    else:
        return 'Other'

# Apply the function to create a new column
combined_data_filtered['event_category'] = combined_data_filtered['events'].apply(categorize_event)

# Calculate the denominator
# Sum up AB, BB, SF, and HBP for each batter-pitcher pair
woba_denominator = combined_data_filtered[combined_data_filtered['event_category'].isin(['AB', 'BB', 'SF', 'HBP'])].groupby(['batter', 'pitcher'])['event_category'].value_counts().unstack(fill_value=0)
woba_denominator['denominator'] = woba_denominator.sum(axis=1)

# Reset the index to make 'batter' and 'pitcher' columns
woba_denominator.reset_index(inplace=True)

In [32]:
woba_numerator = combined_data_filtered.groupby(['batter', 'pitcher'])['woba_value'].sum().reset_index(name='numerator')

# Merge the numerator and denominator dataframes
woba_df = pd.merge(woba_numerator, woba_denominator, on=['batter', 'pitcher'])

# Calculate wOBA
woba_df['wOBA'] = woba_df['numerator'] / woba_df['denominator']

# Handling cases where the denominator is 0 to avoid division by zero errors
woba_df['wOBA'] = woba_df['wOBA'].fillna(0)

woba_df

,batter,pitcher,numerator,AB,BB,HBP,SF,denominator,wOBA
0,405395,425794,0.000,3,0,0,0,3,0.0000
1,405395,425844,0.000,9,0,0,0,9,0.0000
2,405395,448179,2.014,2,0,0,0,2,1.0070
3,405395,453286,1.710,2,0,0,0,2,0.8550
4,405395,456501,0.000,1,0,0,0,1,0.0000
...,...,...,...,...,...,...,...,...,...
64426,807799,676664,0.000,2,0,0,0,2,0.0000
64427,807799,680694,0.855,3,0,0,0,3,0.2850
64428,807799,682243,1.248,3,0,0,0,3,0.4160
64429,807799,686610,3.262,4,0,0,0,4,0.8155


In [33]:
batter_vs_pitcher_woba_df = woba_df.drop(columns = ['numerator', 'AB', 'BB', 'SF', 'HBP', 'denominator'])

In [34]:
batter_vs_pitcher_woba_df

,batter,pitcher,wOBA
0,405395,425794,0.0000
1,405395,425844,0.0000
2,405395,448179,1.0070
3,405395,453286,0.8550
4,405395,456501,0.0000
...,...,...,...
64426,807799,676664,0.0000
64427,807799,680694,0.2850
64428,807799,682243,0.4160
64429,807799,686610,0.8155


In [35]:
# Saving new df off as csv
batter_vs_pitcher_woba_df.to_csv('data/batter_vs_pitcher_woba.zip', index=False)

In [36]:
# Filter the DataFrame to include only rows with 5 or more ABs
filtered_woba_df = woba_df[woba_df['AB'] >= 5]

# Calculate the average wOBA for each batter
average_woba = filtered_woba_df.groupby('batter')['wOBA'].mean().reset_index()

# Find the pitcher against whom each batter had the highest wOBA
max_woba_pitcher = filtered_woba_df.loc[filtered_woba_df.groupby('batter')['wOBA'].idxmax()][['batter', 'pitcher', 'wOBA']]

# Merge the two dataframes
woba_eda_comb = pd.merge(average_woba, max_woba_pitcher, on='batter', suffixes=('_avg', '_max'))

# Sort the dataframe by average wOBA in descending order and select the top 25
top_25_batters = woba_eda_comb.sort_values(by='wOBA_avg', ascending=False).head(25)

# Rename the columns for clarity
top_25_batters.rename(columns={'pitcher': 'fav_pitcher', 'wOBA_avg': 'average_wOBA', 'wOBA_max': 'max_wOBA'}, inplace=True)

# Display the dataframe
top_25_batters

,batter,average_wOBA,fav_pitcher,max_wOBA
516,666181,0.662564,607067,0.883167
122,571718,0.609000,641927,0.609000
521,666310,0.600857,663903,0.600857
441,661531,0.573800,596001,0.573800
663,682848,0.564600,605400,0.564600
388,650968,0.513000,608344,0.513000
105,546990,0.506667,621219,0.671333
42,502210,0.497200,543101,0.573800
601,672515,0.492596,656756,0.994400
103,545361,0.490355,425844,0.982444


In [38]:
# Extract unique player IDs for batters and pitchers
unique_batter_ids = top_25_batters['batter'].unique()
unique_pitcher_ids = top_25_batters['fav_pitcher'].unique()

# Combine and find unique IDs
all_unique_ids = set(list(unique_batter_ids) + list(unique_pitcher_ids))

# Find the names of the players
player_data = playerid_reverse_lookup(list(all_unique_ids), key_type='mlbam')

# Create a mapping from player ID to player name
player_name_mapping = dict(zip(player_data['key_mlbam'], player_data['name_first'] + ' ' + player_data['name_last']))

# Replace the IDs in the DataFrame with names
top_25_batters['batter'] = top_25_batters['batter'].map(player_name_mapping)
top_25_batters['fav_pitcher'] = top_25_batters['fav_pitcher'].map(player_name_mapping)

# Display the updated dataframe
top_25_batters

TypeError: 'zip' object is not callable

### Time to start modeling
Starting with a dummy model and grid iterating new models to improve RMSE

In [39]:
from surprise import Reader, Dataset

In [40]:
reader = Reader(rating_scale=(0, 2.014000))
data = Dataset.load_from_df(batter_vs_pitcher_woba_df,reader)
modeling_data = data.build_full_trainset()
print('Number of users: ', modeling_data.n_users, '\n')
print('Number of items: ', modeling_data.n_items)

Number of users:  918 

Number of items:  258


In [41]:
dummy_model = NormalPredictor()

In [42]:
# Run 5-fold cross-validation and print results
cross_validate(dummy_model, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4467  0.4406  0.4467  0.4460  0.4481  0.4456  0.0026  
MAE (testset)     0.3426  0.3407  0.3441  0.3411  0.3452  0.3427  0.0017  
Fit time          0.03    0.02    0.02    0.02    0.02    0.02    0.00    
Test time         0.27    0.02    0.02    0.02    0.02    0.07    0.10    


{'test_rmse': array([0.44668564, 0.44058965, 0.44665469, 0.44600169, 0.44810003]),
 'test_mae': array([0.34258685, 0.34067764, 0.34412839, 0.34110096, 0.34519936]),
 'fit_time': (0.026257038116455078,
  0.02078986167907715,
  0.02245187759399414,
  0.020679235458374023,
  0.02417159080505371),
 'test_time': (0.2672698497772217,
  0.018513202667236328,
  0.018583059310913086,
  0.018318891525268555,
  0.0186312198638916)}

In [43]:
svd_params = {'n_factors': [20, 50, 100],
              'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD, param_grid=svd_params,n_jobs=-2)
g_s_svd.fit(data)

In [44]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.33782697271113704, 'mae': 0.26108455250475465}
{'rmse': {'n_factors': 20, 'reg_all': 0.1}, 'mae': {'n_factors': 20, 'reg_all': 0.1}}


In [45]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.34356198, 0.33962518, 0.34308273, 0.34991444, 0.34305061]))
('test_mae', array([0.26536551, 0.26237728, 0.26359984, 0.26859732, 0.26515697]))
('fit_time', (0.39843297004699707, 0.42118120193481445, 0.42681384086608887, 0.42162394523620605, 0.4096188545227051))
('test_time', (1.1708860397338867, 1.1193938255310059, 1.1219847202301025, 1.1507346630096436, 1.1439571380615234))
-----------------------
0.34384698970857264


In [46]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [47]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.33582886, 0.34303101, 0.34859192, 0.34114434, 0.34250886]))
('test_mae', array([0.26167614, 0.26449   , 0.26681636, 0.26404547, 0.26364181]))
('fit_time', (0.21656394004821777, 0.2165541648864746, 0.20664095878601074, 0.2164909839630127, 0.2029869556427002))
('test_time', (1.155613899230957, 1.2829082012176514, 1.1785669326782227, 1.165785789489746, 1.1518969535827637))


0.3422209979084962

In [48]:
svdpp = SVDpp()
cv_SVDpp = cross_validate(svdpp, data)
for i in cv_SVDpp.items():
    print(i)    

np.mean(cv_SVDpp['test_rmse'])

('test_rmse', array([0.33952208, 0.34074156, 0.34029565, 0.34157604, 0.33503373]))
('test_mae', array([0.26196178, 0.26306154, 0.2626355 , 0.26260154, 0.25942759]))
('fit_time', (2.092290163040161, 2.081242084503174, 2.095325231552124, 2.0803489685058594, 2.0753390789031982))
('test_time', (0.4408431053161621, 0.43444371223449707, 0.4374349117279053, 0.43496203422546387, 0.5049769878387451))


0.33943381099330266

In [49]:
NMF = NMF()
cv_NMF = cross_validate(NMF,data)
for i in cv_NMF.items():
    print(i)    

np.mean(cv_NMF['test_rmse'])

('test_rmse', array([0.34309701, 0.33928317, 0.34267941, 0.34660574, 0.34240818]))
('test_mae', array([0.25997578, 0.25586431, 0.25759963, 0.25775022, 0.25631313]))
('fit_time', (0.2923910617828369, 0.2714548110961914, 0.29883289337158203, 0.2725951671600342, 0.2967061996459961))
('test_time', (0.023209810256958008, 0.023103952407836914, 0.02373194694519043, 0.022431135177612305, 0.022408008575439453))


0.342814702834909

### SVD++ produced the best test RMSE 
Lets grid search the best hyperparams

In [50]:
svdpp_params1 = {'n_factors': [130, 140, 150],
                'reg_all': [1, 1.1, 1.2],
                'n_epochs': [200, 250, 300],
                'lr_all': [0.001]
}

svdpp_params1 = GridSearchCV(SVDpp, param_grid=svdpp_params1,n_jobs=-1)
svdpp_params1.fit(data)
print(svdpp_params1.best_score)
print(svdpp_params1.best_params)

KeyboardInterrupt: 